In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 08:16:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
input_files="/home/labuser/Downloads/"

In [3]:
df_sales=spark.read.csv(f"{input_files}/sales_data.csv")

In [4]:
df_product=spark.read.csv(f"{input_files}/product_data.csv")

In [5]:
df_sales.show()

+--------------+----------+-----------+-------------+-------------------+
|           _c0|       _c1|        _c2|          _c3|                _c4|
+--------------+----------+-----------+-------------+-------------------+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|
|             1|       101|        201|            5|2023-09-22 10:15:00|
|             2|       102|        202|            3|2023-09-22 11:30:00|
|             3|       101|        203|            2|2023-09-22 12:45:00|
|             4|       103|        204|            1|2023-09-22 14:00:00|
|             5|       102|        205|            4|2023-09-22 15:15:00|
+--------------+----------+-----------+-------------+-------------------+



In [8]:
df_product.show()

+----------+------------+-----------+-----+
|       _c0|         _c1|        _c2|  _c3|
+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
|       101|      Laptop|Electronics|  800|
|       102|  Smartphone|Electronics|  600|
|       103|        Desk|  Furniture|  250|
|       104|  Headphones|Electronics|  100|
|       105|       Chair|  Furniture|  150|
+----------+------------+-----------+-----+



In [7]:
df_sales=spark.read.option("header",True).option("iferschema",True).csv(f"{input_files}/sales_data.csv")

In [9]:
df_product=spark.read.option("header",True).option("iferschema",True).csv(f"{input_files}/product_data.csv")

In [10]:
df = df_sales.join(df_product, df_sales.product_id==df_product.product_id, "inner")
df.show()

+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|product_id|product_name|   category|price|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|             1|       101|        201|            5|2023-09-22 10:15:00|       101|      Laptop|Electronics|  800|
|             2|       102|        202|            3|2023-09-22 11:30:00|       102|  Smartphone|Electronics|  600|
|             3|       101|        203|            2|2023-09-22 12:45:00|       101|      Laptop|Electronics|  800|
|             4|       103|        204|            1|2023-09-22 14:00:00|       103|        Desk|  Furniture|  250|
|             5|       102|        205|            4|2023-09-22 15:15:00|       102|  Smartphone|Electronics|  600|
+--------------+----------+-----------+-------------+-------------------

In [12]:
df.select(df_sales.quantity_sold, df_product.category).show()#this is spark sql

+-------------+-----------+
|quantity_sold|   category|
+-------------+-----------+
|            5|Electronics|
|            3|Electronics|
|            2|Electronics|
|            1|  Furniture|
|            4|Electronics|
+-------------+-----------+



In [13]:
df.filter("transaction_id=1").show()

+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|product_id|product_name|   category|price|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|             1|       101|        201|            5|2023-09-22 10:15:00|       101|      Laptop|Electronics|  800|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+



In [15]:
df.where("transaction_id=1").show()

+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|product_id|product_name|   category|price|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|             1|       101|        201|            5|2023-09-22 10:15:00|       101|      Laptop|Electronics|  800|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+



In [16]:
from pyspark.sql.functions import*

In [17]:
df.where(col("transaction_id")==1).show()

+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|transaction_id|product_id|customer_id|quantity_sold|          timestamp|product_id|product_name|   category|price|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+
|             1|       101|        201|            5|2023-09-22 10:15:00|       101|      Laptop|Electronics|  800|
+--------------+----------+-----------+-------------+-------------------+----------+------------+-----------+-----+



In [18]:
employees = [(1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [19]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [20]:
employeesDF.show()

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [21]:
employeesDF.select(upper("nationality")).show()

+------------------+
|upper(nationality)|
+------------------+
|     UNITED STATES|
|             INDIA|
|    UNITED KINGDOM|
|         AUSTRALIA|
+------------------+



In [28]:
employeesDF = employeesDF.withColumn("ssn_lastFour",split("ssn"," ")[2]).withColumn("country_code",split("phone_number"," ")[0]).withColumn("area_code",split("phone_number"," ")[1])

 

employeesDF.show()

+-----------+----------+---------+------+--------------+----------------+-----------+------------+------------+---------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|ssn_lastFour|country_code|area_code|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+------------+---------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|        6789|          +1|      123|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|        9123|         +91|      234|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|        4444|         +44|      111|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|        6118|         +61|      987|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+------------+---------+



In [30]:
employeesDF.show(truncate=False)#truncate removes white spaces

+-----------+----------+---------+------+--------------+----------------+-----------+------------+------------+---------+
|employee_id|first_name|last_name|salary|nationality   |phone_number    |ssn        |ssn_lastFour|country_code|area_code|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+------------+---------+
|1          |Scott     |Tiger    |1000.0|united states |+1 123 456 7890 |123 45 6789|6789        |+1          |123      |
|2          |Henry     |Ford     |1250.0|India         |+91 234 567 8901|456 78 9123|9123        |+91         |234      |
|3          |Nick      |Junior   |750.0 |united KINGDOM|+44 111 111 1111|222 33 4444|4444        |+44         |111      |
|4          |Bill      |Gomes    |1500.0|AUSTRALIA     |+61 987 654 3210|789 12 6118|6118        |+61         |987      |
+-----------+----------+---------+------+--------------+----------------+-----------+------------+------------+---------+



In [31]:
employeesDF. \
    withColumn("nationality",upper("nationality")).\
    withColumn("ssn_last4", substring(col("ssn"), -4, 4).cast("int")).\
    withColumn("country_code", split("phone_number", " ")[0].cast("int")).\
    withColumn("area_code", split("phone_number", " ")[1].cast("int")).\
    show()

+-----------+----------+---------+------+--------------+----------------+-----------+------------+------------+---------+---------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|ssn_lastFour|country_code|area_code|ssn_last4|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+------------+---------+---------+
|          1|     Scott|    Tiger|1000.0| UNITED STATES| +1 123 456 7890|123 45 6789|        6789|           1|      123|     6789|
|          2|     Henry|     Ford|1250.0|         INDIA|+91 234 567 8901|456 78 9123|        9123|          91|      234|     9123|
|          3|      Nick|   Junior| 750.0|UNITED KINGDOM|+44 111 111 1111|222 33 4444|        4444|          44|      111|     4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|        6118|          61|      987|     6118|
+-----------+----------+---------+------+--------------+----------------+---

In [32]:
spark